In [73]:
#This is avia ticket prediction project!
#first of all, i called all needed libraries and methods which are below

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer

In [95]:
#these dataframes what we use to prepare predictor model, test

train=pd.read_csv('D:/aviachipta-narxini-bashorat-qilish/train_data.csv')
sample=pd.read_csv('D:/aviachipta-narxini-bashorat-qilish/sample_solution.csv', index_col=False)
test=pd.read_csv('D:/aviachipta-narxini-bashorat-qilish/test_data.csv')
train.head(10)

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712
5,6,Air_India,AI-636,Delhi,Afternoon,one,Morning,Chennai,Business,20.42,5,45257
6,7,Vistara,UK-834,Chennai,Evening,one,Morning,Mumbai,Economy,18.42,20,5054
7,8,Air_India,AI-505,Bangalore,Morning,zero,Afternoon,Delhi,Business,2.58,30,32923
8,9,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Hyderabad,Economy,15.92,12,11383
9,10,Air_India,AI-538,Chennai,Night,one,Afternoon,Mumbai,Economy,16.50,18,4357


In [75]:
#Let's check info of the data and whether there is null values or not
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [76]:
# created a copy data of out train data
train_data=train.copy()

In [77]:
# Warn! : I normalized and standarized the train data and saw the correlation between price and the other values,
# and i deleted them not to interrupt
# So this function helps me to drop those lower correlated columns

def waste_data(x):
    return x.drop(['id','flight'], axis=1)

In [78]:
# divided numerical and categorical columns to built pipeline

num_data=['duration','days_left']
cat_data=['airline','source_city','departure_time','stops','arrival_time','destination_city','class']

In [79]:
full_pipline=ColumnTransformer([
    ('numeric',StandardScaler(),num_data),
    ('categorical',OrdinalEncoder(),cat_data)
])

In [80]:
# I split up data into unlabeled and labled data for training
X=train_data.drop('price', axis=1)
y=train_data['price']

In [82]:
#here test_size is 40 % because the amount of data is more, and random state is as usual
X_train, X_test, y_train,y_test=train_test_split(X,y, test_size=0.4, random_state=42)

In [83]:
#Standardizing process
X_train=waste_data(X_train)

X_train_prepared=full_pipline.fit_transform(X_train)

In [84]:
# Traning process
# Warn! : I checked  every predictor models and chose the justified one

xgbr_model=XGBRegressor()
xgbr_model.fit(X_train_prepared, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [85]:
# Preparing the test data for checking the error
x_test_unwasted=waste_data(X_test)
x_test_prepared=full_pipline.fit_transform(x_test_unwasted)

In [86]:
y_predicted=xgbr_model.predict(x_test_prepared)

In [87]:
# the lowest mean absolute error for this data is that you see below
mae=mean_absolute_error(y_test,y_predicted)
msq=np.sqrt(mean_squared_error(y_test,y_predicted))
mae,msq

(2641.9294716701506, 4501.855619899732)

In [88]:
# Preparing the final prediction for test data
test_unwasted=waste_data(test)
test_piplined=full_pipline.fit_transform(test_unwasted)

In [89]:
test_predicted=xgbr_model.predict(test_piplined)

In [98]:
# This is what we should predict
sample['price']=test_predicted
sample.set_index('id', inplace=True)

In [101]:
sample.to_csv('AviaTicketPrediction.csv')